In [1]:
import re
wine_raw_data = pd.read_csv("title_without_winery.csv")

# index 지정
wine_raw_data.rename(columns={"Unnamed: 0":"Index"}, inplace=True)
# 필요없는 컬럼 삭제
wine_raw_data.drop(columns= ['taster_twitter_handle'] , axis=1, inplace=True)
# price랑 country Nan 데이타 제거
wine_raw_data.dropna(subset=['price', 'country'], inplace = True)
wine_raw_data.drop_duplicates(inplace = True)

In [2]:
def get_year(data):
    years = re.findall("\d+", data)
    if len(years) > 0:
        year = sorted([int(year) for year in years], reverse=True)[0]
        return year if year <= 2017 and year >= 1998 else 0
    else:
        return 0
    
wine_raw_data["year"] = wine_raw_data["title"].apply(lambda data: get_year(data))

In [3]:
wine_raw_data= wine_raw_data[wine_raw_data["year"] != 0]

In [4]:
wine1 = wine_raw_data[['Index', 'country', 'points', 'price', 'taster_name', 'variety']]

In [5]:
# taster_name 분류를 위한 문자열 변환
wine1["taster_name"] = wine1["taster_name"].astype('str')

# taster_name 정수형 변환
def mapping(data,feature):
    featureMap=dict()
    count=0
    for i in sorted(data[feature].unique(),reverse=True):
        featureMap[i]=count
        count=count+1
    data[feature]=data[feature].map(featureMap)
    return data

mapping(wine1, "taster_name")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Index,country,points,price,taster_name,variety
1,1,Portugal,87,15.0,4,Portuguese Red
2,2,US,87,14.0,5,Pinot Gris
3,3,US,87,13.0,19,Riesling
4,4,US,87,65.0,5,Pinot Noir
5,5,Spain,87,15.0,7,Tempranillo-Merlot
6,6,Italy,87,16.0,10,Frappato
7,7,France,87,24.0,4,Gewürztraminer
8,8,Germany,87,12.0,18,Gewürztraminer
9,9,France,87,27.0,4,Pinot Gris
10,10,US,87,19.0,1,Cabernet Sauvignon


In [11]:
wine_tasters = []
for i in range (max(wine1["taster_name"])):
    wine_tasters.append(wine1[wine1["taster_name"] == i ])

from sklearn import preprocessing

for wine in wine_tasters:
    wine["norm_points"] = preprocessing.minmax_scale(wine["points"])
    wine["stand_points"] = preprocessing.scale(wine["points"])
    
wine_df = pd.DataFrame(columns=["Index","points", "taster_name"])
while len(wine_tasters) > 0 :
    wine_df = pd.concat([wine_df, wine_tasters[0]])
    del wine_tasters[0]

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [14]:
wine_taster_point= wine_df.sort_values(by="Index")
wine_taster_point

,Index,country,norm_points,points,price,stand_points,taster_name,variety
1,1,Portugal,0.350000,87,15.0,-0.527043,4,Portuguese Red
2,2,US,0.350000,87,14.0,-0.753774,5,Pinot Gris
4,4,US,0.350000,87,65.0,-0.753774,5,Pinot Noir
5,5,Spain,0.388889,87,15.0,0.027770,7,Tempranillo-Merlot
6,6,Italy,0.350000,87,16.0,-0.779290,10,Frappato
7,7,France,0.350000,87,24.0,-0.527043,4,Gewürztraminer
8,8,Germany,0.388889,87,12.0,-0.564909,18,Gewürztraminer
9,9,France,0.350000,87,27.0,-0.527043,4,Pinot Gris
10,10,US,0.368421,87,19.0,-0.744017,1,Cabernet Sauvignon
11,11,France,0.350000,87,30.0,-0.527043,4,Gewürztraminer
